In [ ]:
import os
import requests
import random
import nltk
from textblob import TextBlob
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
CMC_API_KEY = os.getenv("COINMARKETCAP_API_KEY=c9bf995e-4819-43b0-bfc4-396782d7ecb4")

# Download NLTK data if not already present
nltk.download('punkt')

# --- Crypto Intelligence Database (sample) ---
cryptos = {
    "BTC": {
        "name": "Bitcoin",
        "symbol": "BTC",
        "price_trend": "bullish",
        "market_capitalization": "large_cap",
        "energy_consumption_footprint": "high_intensity",
        "sustainability_index": 0.30,
        "volatility_rating": "medium",
        "adoption_score": 0.95,
        "description": "The original cryptocurrency, decentralized and widely adopted."
    },
    "ADA": {
        "name": "Cardano",
        "symbol": "ADA",
        "price_trend": "emerging_growth",
        "market_capitalization": "mid_cap",
        "energy_consumption_footprint": "minimal_intensity",
        "sustainability_index": 0.90,
        "volatility_rating": "high",
        "adoption_score": 0.45,
        "description": "A blockchain platform for changemakers, built on peer-reviewed research."
    }
    # Add more cryptos as needed
}

# --- Live Price Fetching from CoinMarketCap ---
def get_live_price(symbol):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
    headers = {
        "Accepts": "application/json",
        "X-CMC_PRO_API_KEY": "c9bf995e-4819-43b0-bfc4-396782d7ecb4",
    }
    params = {
        "symbol": symbol.upper(),
        "convert": "USD"
    }
    try:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        if "data" not in data or symbol.upper() not in data["data"]:
            print("API error (full response):", data)
            return None
        price = data["data"][symbol.upper()]["quote"]["USD"]["price"]
        return round(price, 4)
    except Exception as e:
        print("API error (exception):", e)
        return None

# --- NLP & Sentiment Analysis ---
def analyze_user_input(user_input):
    blob = TextBlob(user_input)
    sentiment = blob.sentiment.polarity
    lower = user_input.lower()
    # Simple intent/risk/horizon extraction
    if "sustainab" in lower or "green" in lower or "energy" in lower:
        intent = "sustainability"
    elif "profit" in lower or "maximize" in lower:
        intent = "profit"
    else:
        intent = "balanced"
    if "low risk" in lower or "conservative" in lower:
        risk_tolerance = "low"
    elif "moderate" in lower:
        risk_tolerance = "medium"
    else:
        risk_tolerance = "high"
    if "long term" in lower:
        investment_horizon = "long_term"
    else:
        investment_horizon = "short_term"
    return {
        "sentiment": sentiment,
        "intent": intent,
        "risk_tolerance": risk_tolerance,
        "investment_horizon": investment_horizon
    }

# --- Recommendation Logic ---
def recommend_crypto(user_profile):
    intent = user_profile["intent"]
    risk = user_profile["risk_tolerance"]
    horizon = user_profile["investment_horizon"]
    if intent == "sustainability":
        candidates = [c for c in cryptos.values() if c["sustainability_index"] > 0.6]
        if not candidates:
            return None, 0.7
        candidates.sort(key=lambda c: (c["sustainability_index"], c["adoption_score"]), reverse=True)
        crypto = candidates[0]
        confidence = 0.92 if crypto["name"] == "Cardano" else 0.85
    elif intent == "profit":
        candidates = [c for c in cryptos.values() if c["price_trend"] in ["bullish", "emerging_growth"]]
        candidates.sort(key=lambda c: (c["adoption_score"], c["market_capitalization"]), reverse=True)
        crypto = candidates[0]
        confidence = 0.88 if crypto["name"] == "Bitcoin" else 0.80
    else:
        candidates = list(cryptos.values())
        candidates.sort(key=lambda c: (c["adoption_score"], c["sustainability_index"]), reverse=True)
        crypto = candidates[0]
        confidence = 0.80
    return crypto, confidence

# --- Output Formatting with Live Price ---
def print_recommendation(crypto, confidence):
    icons = {
        "bullish": "📈",
        "emerging_growth": "📈",
        "high_intensity": "⚡",
        "minimal_intensity": "🌿",
        "medium": "⚖️",
        "high": "🔥",
        "low": "🟢"
    }
    print(f"\nCryptoAdvisor Pro: Based on comprehensive analysis, {crypto['name']} ({crypto['symbol']}) presents an optimal balance:\n")
    print(f"{icons.get(crypto['price_trend'], '')} Growth Potential: {crypto['price_trend'].replace('_', ' ').title()}")
    print(f"{icons.get(crypto['energy_consumption_footprint'], '')} Sustainability Score: {int(crypto['sustainability_index']*100)}/100")
    print(f"{icons.get(crypto['volatility_rating'], '')} Risk Profile: {crypto['volatility_rating'].capitalize()} volatility")
    print(f"🔗 Adoption Score: {int(crypto['adoption_score']*100)}/100")
    price = get_live_price(crypto['symbol'])
    if price is not None:
        print(f"💰 Current Price: ${price:,.4f}")
    else:
        print("💰 Current Price: (unavailable)")
    print(f"\nRecommendation Confidence: {int(confidence*100)}%")
    print("This analysis is for informational purposes only - please conduct additional research before making investment decisions\n")

# --- CLI Chatbot ---
def chatbot():
    print("Welcome to CryptoAdvisor Pro: Your AI Financial Sidekick!")
    print("I can help you with cryptocurrency investment recommendations.")
    print("Tell me about your goals (e.g., 'sustainable investment', 'maximize profit', 'low risk long-term').")
    print("Type 'exit' or 'quit' to end the session.\n")
    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("CryptoAdvisor Pro: Thank you for using CryptoAdvisor Pro. Happy investing!")
            break
        user_profile = analyze_user_input(user_input)
        crypto, confidence = recommend_crypto(user_profile)
        if crypto:
            print_recommendation(crypto, confidence)
        else:
            print("Sorry, I couldn't find a suitable recommendation based on your criteria.\n")

# --- Run the chatbot ---
if __name__ == "__main__":
    chatbot()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Welcome to CryptoAdvisor Pro: Your AI Financial Sidekick!
I can help you with cryptocurrency investment recommendations.
Tell me about your goals (e.g., 'sustainable investment', 'maximize profit', 'low risk long-term').
Type 'exit' or 'quit' to end the session.


CryptoAdvisor Pro: Based on comprehensive analysis, Bitcoin (BTC) presents an optimal balance:

📈 Growth Potential: Bullish
⚡ Sustainability Score: 30/100
⚖️ Risk Profile: Medium volatility
🔗 Adoption Score: 95/100
💰 Current Price: $105,090.6636

Recommendation Confidence: 80%
This analysis is for informational purposes only - please conduct additional research before making investment decisions


CryptoAdvisor Pro: Based on comprehensive analysis, Bitcoin (BTC) presents an optimal balance:

📈 Growth Potential: Bullish
⚡ Sustainability Score: 30/100
⚖️ Risk Profile: Medium volatility
🔗 Adoption Score: 95/100
💰 Current Price: $105,090.6636

Recommendation Confidence: 80%
This analysis is for informational purposes only - pleas